## Programación Cuadrática
### Optimizacion de carteras con rotación restringida

Este cuaderno plantea el proceso de optimizar una cartera por segunda vez
imponiendo restricciones para que la rotación esté limitada

In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import cvxpy as cp
import pickle

In [2]:
with open('../data/stock_data.pkl', 'rb') as handle:
    stock_data = pickle.load(handle)

Para simplificar el ejercicio trabajaremos con un universo reducido
de 10 activos del IBEX.

In [3]:
tickers = ['ACS','TEF','ITX','GRF','AMS','ENG','MAP','REP','AENA','VIS']

In [4]:
close_dict = {tk: df.close for tk, df in stock_data.items() if tk in tickers}
stock_close = pd.DataFrame(close_dict)

____

**Funcion copiada del ejercicio 3_1**

In [5]:
def efficient_frontier(returns, n_samples=50, gamma_low=-1, gamma_high=10):
    """
    construye un conjunto de problemas de programación cuádrática
    para inferir la frontera eficiente de Markovitz. 
    En cada problema el parámetro gamma se cambia para aumentar
    la penalización del riesgo en la función de maximización.
    """
    sigma = returns.cov().values
    mu = np.mean(returns, axis=0).values  
    n = sigma.shape[0]        
    w = cp.Variable(n)
    gamma = cp.Parameter(nonneg=True)
    ret = mu.T @ w
    risk = cp.quad_form(w, sigma)
    
    prob = cp.Problem(cp.Maximize(ret - gamma*risk), 
                      [cp.sum(w) == 1,  
                       w >= 0,
                       w <= 0.25]) 
    # Equivalente 
    #prob = cp.Problem(cp.Minimize(risk - gamma*ret), 
    #                  [cp.sum(w) == 1,  w >= 0])   
    risk_data = np.zeros(n_samples)
    ret_data = np.zeros(n_samples)
    gamma_vals = np.logspace(gamma_low, gamma_high, num=n_samples)
    
    portfolio_weights = []    
    for i in range(n_samples):
        gamma.value = gamma_vals[i]
        prob.solve()
        risk_data[i] = np.sqrt(risk.value)
        ret_data[i] = ret.value
        portfolio_weights.append(w.value)   
    return ret_data, risk_data, gamma_vals, portfolio_weights



In [6]:
def get_optimal_portfolio(returns):
    ret_data, risk_data, gamma_vals, portfolio_weights = efficient_frontier(returns)
    sharpes = ret_data/risk_data 
    idx = np.argmax(sharpes)
    optimal_portfolio = pd.Series(portfolio_weights[idx],
                              index=returns.columns).round(3)
    return optimal_portfolio

____

#### Optimización Independiente
Primero haremos una optimización por separado para generar 2 carteras cada 6 meses

In [7]:
data_close_h1 = stock_close.loc['2019-01-02':'2019-06-30'].dropna(axis=1)
data_close_h1

,ACS,AMS,AENA,ITX,VIS,MAP,ENG,REP,GRF,TEF
2019-01-02,29.254370,59.910340,131.210901,21.723158,44.423518,2.062509,21.491329,11.817150,22.446920,6.995052
2019-01-03,29.324778,56.614491,131.258788,21.413242,44.290456,2.072394,21.759635,11.833753,22.525924,7.111227
2019-01-04,29.764826,58.564697,134.563004,22.168662,45.088825,2.130810,21.804353,12.116003,23.276459,7.203789
2019-01-07,29.861636,59.110755,133.270050,22.342990,45.240895,2.129911,21.786465,12.016386,23.256708,7.268960
2019-01-08,30.169670,60.612414,134.084132,22.827233,46.134308,2.127215,22.090546,12.028838,23.602349,7.257626
...,...,...,...,...,...,...,...,...,...,...
2019-06-24,33.680991,67.207544,177.350000,25.096468,44.728066,2.446751,22.278360,12.413808,24.620179,7.046115
2019-06-25,33.486250,66.853510,177.300000,24.850327,44.902785,2.444895,22.099489,12.288867,24.739405,7.065531
2019-06-26,32.215794,67.128870,172.650000,25.106313,44.262148,2.431905,21.840127,12.400422,24.173081,7.083976
2019-06-27,32.215794,67.561579,172.050000,25.903808,44.514520,2.396647,21.500272,12.235321,25.047406,7.003400


In [8]:
returns_h1 = np.log(data_close_h1).diff().dropna()
returns_h1.head()

,ACS,AMS,AENA,ITX,VIS,MAP,ENG,REP,GRF,TEF
2019-01-03,0.002404,-0.056584,0.000365,-0.014369,-0.003000,0.004782,0.012407,0.001404,0.003513,0.016472
2019-01-04,0.014895,0.033867,0.024862,0.034670,0.017865,0.027797,0.002053,0.023571,0.032776,0.012932
2019-01-07,0.003247,0.009281,-0.009655,0.007833,0.003367,-0.000422,-0.000821,-0.008256,-0.000849,0.009006
2019-01-08,0.010263,0.025087,0.006090,0.021442,0.019555,-0.001267,0.013861,0.001036,0.014753,-0.001560
2019-01-09,0.008713,0.004494,-0.010772,0.011390,0.004522,-0.004658,-0.004463,0.014049,0.013713,-0.020644


portfolio optimo para el primer semestre

In [9]:
portfolio_h1 = get_optimal_portfolio(returns_h1)
portfolio_h1

ACS     0.099
AMS     0.048
AENA    0.250
ITX     0.241
VIS     0.000
MAP     0.250
ENG    -0.000
REP    -0.000
GRF     0.112
TEF    -0.000
dtype: float64

ahora para el segundo semestre

In [10]:
data_close_h2 = stock_close.loc['2019-07-01':'2019-12-31'].dropna(axis=1)
returns_h2 = np.log(data_close_h2).diff().dropna()
returns_h2.head()

,ACS,AMS,AENA,ITX,VIS,MAP,ENG,REP,GRF,TEF
2019-07-02,0.004755,-0.004208,0.009406,0.003020,-0.002151,0.013326,0.025603,-0.011058,0.004957,0.004801
2019-07-03,0.025074,0.001966,0.005093,0.012734,0.009003,0.006785,0.007257,-0.011907,0.017346,0.001641
2019-07-04,-0.004637,0.001682,0.002255,0.007785,0.000427,0.007858,-0.043911,-0.000363,0.012999,0.014646
2019-07-05,-0.011825,-0.009569,-0.023940,0.004422,-0.005991,-0.001119,-0.074724,-0.002181,0.026581,-0.004588
2019-07-08,0.018093,0.010130,0.000865,-0.000368,-0.002148,-0.001494,-0.049329,0.001091,0.002870,-0.001083


In [11]:
portfolio_h2 = get_optimal_portfolio(returns_h2)
portfolio_h2

ACS     0.000
AMS     0.094
AENA    0.000
ITX     0.250
VIS    -0.000
MAP     0.000
ENG     0.156
REP     0.250
GRF     0.250
TEF     0.000
dtype: float64

___

Miramos el resultado semestral

In [12]:
result_h2 = data_close_h2.iloc[-1]/data_close_h2.iloc[0]
result_h2

ACS     0.999439
AMS     1.028526
AENA    0.976518
ITX     1.207681
VIS     1.025729
MAP     0.927351
ENG     1.028879
REP     1.049129
GRF     1.209252
TEF     0.881927
dtype: float64

Calculamos propocionalmente, como si tuvieramos una cartera de 1€

In [13]:
port_res = result_h2 * portfolio_h1
port_res

ACS     0.098944
AMS     0.049369
AENA    0.244129
ITX     0.291051
VIS     0.000000
MAP     0.231838
ENG    -0.000000
REP    -0.000000
GRF     0.135436
TEF    -0.000000
dtype: float64

La cartera de H1 a final de año tendría los siguientes pesos

In [14]:
port1_ath2 = port_res/port_res.sum()
port1_ath2

ACS     0.094164
AMS     0.046984
AENA    0.232334
ITX     0.276989
VIS     0.000000
MAP     0.220636
ENG    -0.000000
REP    -0.000000
GRF     0.128893
TEF    -0.000000
dtype: float64

**Diferencia** respecto a la nueva asignación de la cartera h2

In [15]:
portfolio_h2 - port1_ath2

ACS    -0.094164
AMS     0.047016
AENA   -0.232334
ITX    -0.026989
VIS    -0.000000
MAP    -0.220636
ENG     0.156000
REP     0.250000
GRF     0.121107
TEF     0.000000
dtype: float64

**Rotación de la cartera** 

In [16]:
rotacion = (portfolio_h2 - port1_ath2).abs().sum()
rotacion

1.1482469859465643

____

#### Optimización con restricciones de rotación

In [19]:
def efficient_frontier_max_rotation(returns, current_port,
                                    max_rotation=0.8,  
                                    n_samples=50, 
                                    gamma_low=-1, gamma_high=5):
    sigma = returns.cov().values
    mu = np.mean(returns, axis=0).values  
    n = sigma.shape[0]        
    
    w = cp.Variable(n)
         
    gamma = cp.Parameter(nonneg=True)
    ret = mu.T @ w
    risk = cp.quad_form(w, sigma)
    
    constraints = [
        cp.sum(w) == 1,  
        w >= 0,
        w <= 0.25,
        
        # restriccion para que la rotacion este limitada
        cp.sum(cp.abs(w - current_port.values)) <= max_rotation,
    ]
    
    prob = cp.Problem(cp.Maximize(ret - gamma*risk), constraints) 
    
    risk_data = np.zeros(n_samples)
    ret_data = np.zeros(n_samples)
    gamma_vals = np.logspace(gamma_low, gamma_high, num=n_samples)
    
    portfolio_weights = []
    buys_wg = []
    sells_wg = []
    for i in range(n_samples):
        gamma.value = gamma_vals[i]
        prob.solve(solver='ECOS')
        risk_data[i] = np.sqrt(risk.value)
        ret_data[i] = ret.value
        portfolio_weights.append(w.value)  
        
    return ret_data, risk_data, gamma_vals, portfolio_weights, buys_wg, sells_wg


In [20]:
ret_data, risk_data, gamma_vals, portfolio_weights, port_buys, port_sells = efficient_frontier_max_rotation(
    returns_h2, port1_ath2, max_rotation=0.5
)

sharpes = ret_data/risk_data 
idx = np.argmax(sharpes)
portfolio_h2_rotation = pd.Series(portfolio_weights[idx],
                                  index=returns_h2.columns).round(3)
portfolio_h2_rotation

ACS     0.092
AMS     0.047
AENA    0.232
ITX     0.250
VIS     0.000
MAP     0.000
ENG     0.061
REP     0.068
GRF     0.250
TEF     0.000
dtype: float64

In [21]:
portfolio_h2_rotation - port1_ath2

ACS    -0.002164
AMS     0.000016
AENA   -0.000334
ITX    -0.026989
VIS     0.000000
MAP    -0.220636
ENG     0.061000
REP     0.068000
GRF     0.121107
TEF     0.000000
dtype: float64

**Rotación limitada** 

In [22]:
rotacion2 = (portfolio_h2_rotation - port1_ath2).abs().sum()
rotacion2

0.5002469859465642

In [24]:

res = pd.concat([port1_ath2, portfolio_h2_rotation, portfolio_h2], axis=1)
res.columns = ['current','rotacion_limitada','portfolio_independiente']
res

,current,rotacion_limitada,portfolio_independiente
ACS,0.094164,0.092,0.000
AMS,0.046984,0.047,0.094
AENA,0.232334,0.232,0.000
ITX,0.276989,0.250,0.250
VIS,0.000000,0.000,-0.000
MAP,0.220636,0.000,0.000
ENG,-0.000000,0.061,0.156
REP,-0.000000,0.068,0.250
GRF,0.128893,0.250,0.250
TEF,-0.000000,0.000,0.000
